## ETL DIM_STAFF

In [98]:
import pandas as pd
from config.db_config import conn, source

### EXT

In [99]:
df_staff = pd.read_sql_query("select * from staff", source)
df_store = pd.read_sql_query("select * from store", source)

### TRA

In [100]:
merge_df = pd.merge(df_staff, df_store, how='inner', on='store_id')
merge_df["name"] = merge_df['first_name'] + " " + merge_df['last_name']
merge_df = merge_df[['name', 'store_id']]
merge_df['id_staff'] = range(1, len(merge_df) + 1)
df_staff = merge_df[['id_staff', 'name', 'store_id']]
df_staff = df_staff.rename(columns = {"store_id": "store"})
df_staff['name'] = (df_staff['name'].astype(str)).str.title()
df_staff['store'] = (df_staff['store'].astype(str)).str.title()

### CAR

In [101]:
cursor = conn.cursor()
try:
    insert_query = """
        INSERT INTO dim_staff (id_staff, name, store) 
        VALUES (%s, %s, %s)
        ON CONFLICT (id_staff) DO UPDATE
        SET name = EXCLUDED.name, store = EXCLUDED.store;
        """
    filas = 0
    for index, row in df_staff.iterrows():
        values = tuple(row)
        cursor.execute(insert_query, values)
        filas += 1
    conn.commit()
    print(f"Se afectaron {filas} filas exitosamente.")
except Exception as e:
    conn.rollback()
    print("Error durante la insercion:", e)
finally:
    cursor.close()

Se afectaron 2 filas exitosamente.
